In [ ]:
import pygame
from pygame.locals import *
import sys
import random
import math

class Agent:
    def __init__(self,x,y,vx,vy):
        self.x, self.y = x,y
        self.vx,self.vy = vx, vy
    
    def update(self, WINDOW_W, WINDOW_H):
        self.x += self.vx
        self.y += self.vy
        if self.x > WINDOW_W:
            self.vx, self.x = -self.vx, WINDOW_W
            return
        if self.x < 0:
            self.vx, self.x = -self.vx, 0
            return
        if self.y > WINDOW_H:
            self.vy, self.y = -self.vy, WINDOW_H
            return
        if self.y < 0:
            self.vy, self.y = -self.vy, 0
            return
        
    def draw(self,screen):
        x, y = int(self.x), int(self.y)
        #pygame.draw.circle(Surface, color, pos, radius, width=0): return Rect
        pygame.draw.circle(screen,(255,0,0),(x,y),5)
        
    def separation(self, r_s):#分離のルール
        tvx = tvy = c = 0
        for a in self.others:#自分以外の全エージェントを処理する
            if a[1] < r_s and a[1] != 0:#近すぎるか？a[1]は距離
                tvx -= (a[0].x - self.x) / a[1]
                tvy -= (a[0].y - self.y) / a[1]
                c += 1
        if c != 0:
            self.vx_s, self.vy_s = tvx/c, tvy/c
        else:
            self.vx_s, self.vy_s = self.vx, self.vy
            
    def alignment(self, r_a):#整列のルール
        tvx = tvy = c = 0
        for a in self.others:#自分以外の全エージェントを処理する
            if a[1] < r_a:#周囲のエージェントか？a[1]は距離
                tvx += a[0].vx
                tvy += a[0].vy
                c += 1
        if c != 0:
            self.vx_a, self.vy_a = tvx/c, tvy/c
        else:
            self.vx_a, self.vy_a = self.vx, self.vy
            
    def cohesion(self, r_c):
        tx = ty = c = 0
        for a in self.others:#自分以外の全エージェントを処理する
            if a[1] < r_c:#周囲のエージェントか？a[1]は距離
                tx += a[0].x
                ty += a[0].y
                c += 1
        if c != 0:
            tx, ty= tx/c, ty/c#重心を求める
            d = math.sqrt((tx - self.x)**2 + (ty - self.y)**2)
            if d != 0:
                self.vx_c = (tx - self.x)/d
                self.vy_c = (ty - self.y)/d
        else:
            self.vx_c, self.vy_c = self.vx, self.vy
            
    def rule(self, agent_list, r_s, r_a, r_c):
        #他エージェントとの距離を求める
        self.others = tuple([(a, math.sqrt((a.x - self.x)**2 + (a.y - self.y)**2))for a in agent_list if a!=self])
        if len(self.others) < 1: return
        
        #3つのルールを適用
        self.separation(r_s)
        self.alignment(r_a)
        self.cohesion(r_c)
        
        tvx = self.vx_s * 1.0 + self.vx_a * 0.4 + self.vx_c * 0.2
        tvy = self.vy_s * 1.0 + self.vy_a * 0.4 + self.vy_c * 0.2
        n = math.sqrt(tvx**2 + tvy**2)
        #新しい速度を設定
        self.vx, self.vy = 2 * tvx / n, 2 * tvy / n
    
WINDOW_W, WINDOW_H = 1000, 700
pygame.init() #pygameを初期化
screen = pygame.display.set_mode((WINDOW_W, WINDOW_H)) #画面を作成
clock = pygame.time.Clock() #時間を管理するためのClockオブジェクトを作成
font = pygame.font.Font(None,28)
agent_list = []
rule_on = 1

while True:
    clock.tick(60) #60fpsに設定
    screen.fill((255,255,255))#白色
    
    for a in agent_list: #各エージェントを処理する
        if rule_on == 1:
            a.rule(agent_list, 15, 30, 50)
        a.update(WINDOW_W, WINDOW_H)
        a.draw(screen)
        
    #エージェントの数を表示
    s1 ="agents : " + str(len(agent_list))
    text = font.render(s1, True, (0,0,0))
    screen.blit(text, (1,1))
    
    #イベントを処理する
    for e in pygame.event.get():
        #マウスの左ボタンが押された時の処理
        if e.type==MOUSEBUTTONDOWN and e.button==1:
            #速度を乱数で設定
            vx = random.uniform(-1.5, 1.5)
            vy = random.uniform(-1.5, 1.5)
            #エージェントを生成
            a = Agent(e.pos[0],e.pos[1],vx,vy)
            #生成したエージェントをagent_listに追加
            agent_list.append(a)
        if e.type==MOUSEBUTTONDOWN and e.button==2:
            rule_on = 1-rule_on #フラグの切り替え
        if e.type==QUIT:
            pygame.quit()
            sys.exit()
        pygame.display.update()
        